### 장르 속성을 이용한 영화 콘텐츠 기반 필터링

데이터 로딩 및 가공

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

movies = pd.read_csv('dataset/tmdb_5000_movies.csv')
print(movies.shape)
movies.head(2)

# 4803개의 레코드와 20개의 피처로 구성되어 있다.
# 예산, 홈페이지, 영화제목, 개요, 인기도, 평점, 투표 수, 예산, 키워드 등 영화에 대한 다양한 메타 정보를 가지고 있다.
# 여기서 콘텐츠 기반 필터링 추천 분석에 사용할 주요 칼럼만 추출해 새롭게 DataFrame으로 만든다.

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


- genres, keywords 같은 칼럼은 리스트 내부에 여러 개의 딕셔너리(dict)가 있는 형태의 문자열로 표기되어 있다.
    - [{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}]와 같은 형태
- 이는 한꺼번에 여러 개의 값을 표현하기 위한 표기 방식이다.
- 예를 들어 영화 아바타의 genres는 'Action'과 'Adventure'등의 여러 가지 장르로 구성될 수 있기 때문이다.
- 하지만 이 칼럼이 DataFrame으로 만들어질 때는 단순히 문자열 형태로 로딩되므로 이 칼럼을 가공하지 않고는 필요한 정보를 추출할 수가 없다.

추출할 주요 칼럼 
- id 
- 영화제목 title
- 영화가 속한 여러 가지 장르 genres
- 평균 평점 vote_average
- 평점 투표 수 vote_count
- 영화의 인기 popularity
- 영화를 설명하는 주요 키워드 문구 keywords
- 영화에 대한 개요 설명 overview

In [2]:
movies_df = movies[['id', 'title', 'genres', 'vote_average', 'vote_count', 'popularity', 'keywords', 'overview']]

In [3]:
pd.set_option('max_colwidth',100)
movies_df[['genres','keywords']][:1]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp..."


- 위와 같이 genres 칼럼은 여러 개의 개별 장르 데이터를 가지고 있고, 이 개별 장르의 명칭은 딕셔너리의 키(Key)인 'name'으로 추출할 수 있다.
- Keywords 역시 마찬가지 구조를 가지고 있다.
- genres 칼럼의 문자열을 분해해서 개별 장르를 파이썬 리스트 객체로 추출한다.
- 파이썬 ast 모듈의 literal_eval()함수를 이용하면 이 문자열을 문자열이 의미하는 list [dict1, dict2]객체로 만들 수 있다.
- Series 객체의 apply()에 literal_eval 함수를 적용해 문자열을 객체로 변환한다.

In [4]:
# genres 칼럼은 문자열로 표기되어 있으며 문자열을 분해해서 개별 장르를 파이썬 리스트 객체로 추출
from ast import literal_eval

movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

- genres 칼럼은 여러 개의 개별 장르 데이터를 가지고 있고, 이 개별 장르의 명칭은 딕셔너리의 key인 'name'으로 추출할 수 있다.
- Keywords 역시 마찬가지 구조를 가진다.
- genres 칼럼의 문자열을 분해해서 개별 장르를 파이썬 리스트 객체로 추출한다.
- 파이썬 ast 모듈의 literal_eval()함수를 이용하면 이 문자열을 문자열이 의미하는 list[dict1, dict2] 객체로 만들 수 있다.

In [5]:
# 이제 genres 칼럼은 문자열이 아니라 실제 리스트 내부에 여러 장르 딕셔너리로 구성된 객체를 가진다.
# genres라는 dict형태에서 'name' key의 value들만 가져오기
movies_df['genres'] = movies_df['genres'].apply(lambda x : [y['name'] for y in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [y['name'] for y in x])
movies_df[['genres', 'keywords']][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa..."


In [ ]:
# 위에 코드 합친것 
from ast import literal_eval
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')
movies = pd.read_csv('dataset/tmdb_5000_movies.csv', encoding='utf-8')

# 컨텐츠를 추천하는 시스템을 만들기 위해서 해당 영화들의 유사도 비교할 항목들만 추출
movies_df = movies[['id','title', 'genres', 'vote_average', 'vote_count',
                 'popularity', 'keywords', 'overview']]

movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

# genres라는 dict형태에서 'name' key의 value들만 가져오기
movies_df['genres'] = movies_df['genres'].apply(lambda x: [y['name'] for y in x])
# keywords도 동일하게 수행
movies_df['keywords'] = movies_df['keywords'].apply(lambda x: [y['name'] for y in x])

# 리스트에 담겨있는 genres, keywords를 공백 기준으로 문자열로 붙이기
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : (' ').join(x))
movies_df['keywords_literal'] = movies_df['keywords'].apply(lambda x: (' ').join(x))

장르 콘텐츠 유사도 측정
- 문자열로 변환된 genres 칼럼을 Count 기반으로 피처 벡터화 변환한다.
- genres 문자열을 피처 벡터화 행렬로 변환한 데이터 세트를 코사인 유사도를 통해 비교한다.
- 이를 위해 데이터 세트의 레코드별로 타 레코드와 장르에서 코사인 유사도 값을 가지는 객체를 생성한다.
- 장르 유사도가 높은 영화 중에 평점이 높은 순으로 영화를 추천한다.

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

# 리스트 객체 값으로 구성된 genres 칼럼을 apply(lambda x :('').join(x))를 적용하여
# 개별 요소를 공백 문자로 구분하는 문자열로 변환해 genres_listeral 칼럼으로 저장
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : (' ').join(x))
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
print(genre_mat.shape)

(4803, 276)


- CountVectorizer로 변환해 4803개의 레코드와 276개의 개별 단어 피처로 구성된 피처 벡터 행렬을 생성
- 이렇게 생성된 피처 벡터 행렬에 사이킷런의 cosine_similarity()를 이용해 코사인 유사도 계산

In [9]:
# 피처 벡터화된 행렬에 cosine_similairities()를 적용
# 반환된 코사인 유사도 행렬의 크기 및 앞 2개 데이터만 추출
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:1])


(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]]


- cosine_similarity() 호출로 생성된 genre_sim 객체는 movies_df의 genre_literal 칼럼을 피처 벡터화한 행렬(genre_mat) 데이터 행(레코드)별 유사도 정보를 가지고 있으며, 결국은 movies_df DataFrame의 행별 장르 유사도 값을 가지고 있다.
- mobies_df를 장르 기준으로 콘텐츠 기반 필터링을 수행하려면 movies_df의 개별 레코드에 대해서 가장 장르 유사도가 높은 순으로 다른 레코드를 추출해야 하는데, 이를 위해 앞에서 생성한 genre_sim 객체를 이용한다.
- genre_sim 객체의 기준 행별로 비교 대상이 되는 행의 유사도 값이 높은 순으로 정렬된 행렬의 위치 인덱스 값을 추출하면 된다. 값이 높은 순으로 정렬된 비교 대상 행의 유사도 값이 아니라 비교 대상 행의 위치 인덱스임에 주의한다.

In [10]:
# 높은 순으로 정렬된 비교 행 위치 인덱스 값을 가져오고 그중에 0번 레코드의 비교 행 위치 인덱스 값만 샘플로 추출
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]
print(genre_sim_sorted_ind[:1])

# 자기 자신인 0번 레코드를 제외하고 그 다음으로 높은 3494 ... 제일 유사도가 낮은 2401번 레코드까지 추출

[[   0 3494  813 ... 3038 3037 2401]]


장르 콘텐츠 필터링을 이용한 영화 추천
- find_sim_movie 인자로 movies_df DataFrame, 레코드별 장르 코사인 유사도 인덱스, genre_sim_sorted_ind, 고객이 선정한 추천 기준이 되는 영화제목, 추천할 영화 건수

In [16]:
# 장르 유사도에 따라 영화를 추천하는 함수
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    
    # 인자로 입력된 movies_df DataFrame에서 'title' 칼럼이 입력된 title_name 값인 DataFrame 추출
    title_movie = df[df['title'] == title_name]
    
    # title_named을 가진 DataFrame의 index 객체를 ndarray로 반환하고
    # sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n개의 indext 추출
    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]
    
    # 추출된 top_n index 출력, top_n index는 2차원 데이터
    # dataframe에서 idnex로 사용하기 위해서 1차원 array로 변경
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    
    return df.iloc[similar_indexes]

In [17]:
# The Godfather와 장르별로 유사한 영화 10개 추천
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
similar_movies[['title', 'vote_average']]

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866]]


,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1


- The Godfather: Part II 가장 먼저 추천 그 외에 GoodFellas도 The Godfather와 유사하다. 
- 하지만 Mi America, Light Sleeper 등 평점이 낮고 0점인 경우도 있어서 좀 더 개선이 필요하다.
- 좀 더 많은 후보군을 선정한 뒤에 영화의 평점에 따라 필터링해서 최종 추천하는 방식으로 변경한다.
- 영화 평점 정보인 'vote_average' 값을 이용
    - 적용시 유의할 점
        - 0부터 10점 만점까지의 점수로 되어 있는데, 여러 관객이 평가한 평점을 평균한 것이다.
        - 그런데 1명, 2명의 소수의 관객이 특정 영화에 만점이나 매우 높은 평점을 부여해 왜곡된 데이터를 가지고 있다.
        - 이를 확인하기 위해서 sort_values()를 이용해 평점('vote_average') 오름차순으로 movies_df를 정렬해서 10개만 출력한다.

In [21]:
# moviesd_df의 tilte, vote_averge, voet_count를 정렬하는데 vote_average를 기준으로 내림차순
movies_df[['title','vote_average', 'vote_count']].sort_values('vote_average', ascending=False)[:10]

# 출력결과 왜곡된 데이터 확인 듣도보도 못한 영화 vote_count 1인데 평점 10 -> 평가 횟수가 매우 작다.
# 이와 같은 왜곡된 평점 데이터를 회피할 수 있도록 평점에 평가 횟수를 반영할 수 있는 새로운 평가 방식이 필요하다.

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


<img src='dataset/weighted_rating.png'>

- V는 movies_df의 'vote_count'값
- R 값은 'vote_average'값
- C의 경우 전체 영화의 평균 평점 -> movies_df['vote_average'].mean()
- m의 경우는 투표 횟수에 따른 가중치를 직접 조절하는 역할을 하는데, m 값을 높이면 평점 투표 횟수가 많은 영화에 더 많은 가중 평점을 부여한다.
    - m 값은 전체 투표 횟수에서 상위 60% 해당하는 횟수를 기준으로 정한다.
    - 상위 60%값은 Series 객체의 quantile()을 이용해 추출

In [23]:
C = movies_df['vote_average'].mean()
m = movies_df['vote_count'].quantile(0.6)
print('C:', round(C,3),'m:', round(m,3))

C: 6.092 m: 370.2


- 기존 평점을 새로운 가중 평점으로 변경하는 함수를 생성하고 이를 이용해 새로운 평점 정보인 'vote_weighted'값을 만든다.
- 함수 명은 weighted_vote_average()
- 이 함수는 DataFrame의 레코드를 인자로 받아 이 레코드의 vote_count와 vote_average 칼럼, 그리고 미리 추출된 m과 C값을 적용해 레코드별 가중 평점을 반환한다.

In [29]:
percentile = 0.6
m = movies['vote_count'].quantile(percentile)
C = movies['vote_average'].mean()

def weighted_vote_average(record):
    v = record['vote_count']
    R = record['vote_average']
    
    return ( (v/(v+m)) * R) + ( (m/(m+v)) * C)

movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis=1)

In [30]:
movies_df[['title', 'vote_average', 'weighted_vote', 'vote_count']].sort_values('weighted_vote', ascending=False)[:10]

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


In [38]:
# 가중 평점 변수를 추가해서 유사한 영화 찾아주는 함수 재정의
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values
    
    # top_n의 2배에 해당하는 장르 유사성이 높은 인덱스 추출
    similar_indexes = sorted_ind[title_index, :(top_n*2)]
    similar_indexes = similar_indexes.reshape(-1,)
    
    # 자기 자신 영화 제외, boolean index기법 사용
    similar_indexes = similar_indexes[similar_indexes != title_index]
    
    # top_n의 2배에 해당하는 후보군에서 weighted_vote가 높은 순으로 top_n만큼 추출
    return df.iloc[similar_indexes].sort_values('weighted_vote', ascending=False)[:top_n]

similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather' , 10)
similar_movies[['title', 'vote_average', 'weighted_vote']]
                                 

,title,vote_average,weighted_vote
2731,The Godfather: Part II,8.3,8.079586
1847,GoodFellas,8.2,7.976937
3866,City of God,8.1,7.759693
1663,Once Upon a Time in America,8.2,7.657811
883,Catch Me If You Can,7.7,7.557097
281,American Gangster,7.4,7.141396
4041,This Is England,7.4,6.739664
1149,American Hustle,6.8,6.717525
1243,Mean Streets,7.2,6.626569
2839,Rounders,6.9,6.530427
